Cleaning , pre-processing , Feature engineering

In [1]:
from datetime import datetime
import re
import pandas as pd
import os

# Column names for the structured log
colz = ["IP", "timestamp", "method", "URL", "protocol", "status code", "bytes sent", "user agent"]

# Configure processing
input_file = "access.log"
output_file = "cleaned_access_log.csv"
chunk_size = 100000  # Process 100k lines at a time
error_log = "processing_errors.log"


def process_line(line):
    try:
        parts = line.split()
        if len(parts) < 12:
            return None
        
        # Handle bytes sent (can be '-')
        try:
            bytes_sent = int(parts[9])
        except ValueError:
            bytes_sent = 0  # Default value for missing bytes
            
        # Handle status code
        try:
            status_code = int(parts[8])
        except ValueError:
            status_code = 0  # Default value for invalid status
            
        return [
            parts[0],                       # IP
            parts[3].strip('['),            # Timestamp
            parts[5].strip('"'),            # Method
            parts[6],                       # URL
            parts[7].strip('"'),            # Protocol
            status_code,                    # Status code
            bytes_sent,                     # Bytes sent
            ' '.join(parts[11:]).strip('"') # User agent
        ]
    except Exception as e:
        with open(error_log, 'a') as err:
            err.write(f"Error processing line: {line.strip()}\nError: {str(e)}\n\n")
        return None

def save_chunk(data):
    logs_df = pd.DataFrame(data, columns=colz)
    
    # Parse timestamp
    logs_df["timestamp"] = pd.to_datetime(
        logs_df["timestamp"], 
        format='%d/%b/%Y:%H:%M:%S', 
        errors='coerce'
    )
    
    # Feature engineering 
    logs_df["url_length"] = logs_df["URL"].apply(len)
    logs_df["url_depth"] = logs_df["URL"].apply(lambda x: x.count('/'))
    logs_df['num_encoded_chars'] = logs_df['URL'].apply(lambda x: len(re.findall(r'%[0-9A-Fa-f]{2}', x)))
    logs_df['num_special_chars'] = logs_df['URL'].apply(lambda x: len(re.findall(r'[|,;]', x)))
    
    # Save chunk
    logs_df.to_csv(
        output_file, 
        mode='a', 
        header=not os.path.exists(output_file), 
        index=False
    )

# Process file in chunks
current_chunk = []
processed_lines = 0
successful_lines = 0

with open(input_file) as f:
    for line in f:
        processed_lines += 1
        if processed_lines % 100000 == 0:
            print(f"Processed {processed_lines} lines...")
            
        parsed = process_line(line)
        if parsed:
            current_chunk.append(parsed)
            successful_lines += 1
            
        # Process and save chunk when reaching chunk size
        if len(current_chunk) >= chunk_size:
            save_chunk(current_chunk)
            current_chunk = []

    # Process any remaining lines in the final chunk
    if current_chunk:
        save_chunk(current_chunk)
        
print(f"\nProcessing complete!")
print(f"Total lines processed: {processed_lines}")
print(f"Successfully parsed lines: {successful_lines}")
print(f"Error rate: {(processed_lines-successful_lines)/processed_lines:.2%}")

Processed 100000 lines...
Processed 200000 lines...
Processed 300000 lines...
Processed 400000 lines...
Processed 500000 lines...
Processed 600000 lines...
Processed 700000 lines...
Processed 800000 lines...
Processed 900000 lines...
Processed 1000000 lines...
Processed 1100000 lines...
Processed 1200000 lines...
Processed 1300000 lines...
Processed 1400000 lines...
Processed 1500000 lines...
Processed 1600000 lines...
Processed 1700000 lines...
Processed 1800000 lines...
Processed 1900000 lines...
Processed 2000000 lines...
Processed 2100000 lines...
Processed 2200000 lines...
Processed 2300000 lines...
Processed 2400000 lines...
Processed 2500000 lines...
Processed 2600000 lines...
Processed 2700000 lines...
Processed 2800000 lines...
Processed 2900000 lines...
Processed 3000000 lines...
Processed 3100000 lines...
Processed 3200000 lines...
Processed 3300000 lines...
Processed 3400000 lines...
Processed 3500000 lines...
Processed 3600000 lines...
Processed 3700000 lines...
Processed 

Reading from the new dataset and getting info

In [2]:
# Calculate date range if we processed any lines
if successful_lines > 0:
    logs_df = pd.read_csv(output_file, parse_dates=['timestamp'])
    min_date = logs_df['timestamp'].min()
    max_date = logs_df['timestamp'].max()
    date_span = max_date - min_date

# Basic DataFrame info
print("=== SHAPE ===")
print(f"Rows: {logs_df.shape[0]}, Columns: {logs_df.shape[1]}\n")

print("=== COLUMNS ===")
print(logs_df.columns.tolist())
print("\n")

print("=== DATA TYPES ===")
print(logs_df.dtypes)
print("\n")

print("=== SAMPLE DATA ===")
print(logs_df.head())  # First 5 rows
print("\n")

print("=== SUMMARY STATS ===")
print(logs_df.describe(include='all'))  # For both numeric and categorical
print("\n")

print("=== MISSING VALUES ===")
print(logs_df.isnull().sum())
print("\n")

print("=== UNIQUE COUNTS ===")
print(logs_df.nunique())

print(f"\nDate Range:")
print(f"Min Date: {min_date}")
print(f"Max Date: {max_date}")
print(f"Date Span: {date_span}")


=== SHAPE ===
Rows: 10365109, Columns: 12

=== COLUMNS ===
['IP', 'timestamp', 'method', 'URL', 'protocol', 'status code', 'bytes sent', 'user agent', 'url_length', 'url_depth', 'num_encoded_chars', 'num_special_chars']


=== DATA TYPES ===
IP                           object
timestamp            datetime64[ns]
method                       object
URL                          object
protocol                     object
status code                   int64
bytes sent                    int64
user agent                   object
url_length                    int64
url_depth                     int64
num_encoded_chars             int64
num_special_chars             int64
dtype: object


=== SAMPLE DATA ===
              IP           timestamp method  \
0   54.36.149.41 2019-01-22 03:56:14    GET   
1    31.56.96.51 2019-01-22 03:56:16    GET   
2    31.56.96.51 2019-01-22 03:56:16    GET   
3  40.77.167.129 2019-01-22 03:56:17    GET   
4    91.99.72.15 2019-01-22 03:56:17    GET   

        